### Library

In [ ]:
# =======================
# Built-in / Utilities
# =======================
import os
import time
import random
import warnings
from collections import Counter

warnings.filterwarnings('ignore')

# =======================
# Core Libraries
# =======================
import numpy as np
import pandas as pd

# =======================
# Visualization
# =======================
import matplotlib.pyplot as plt
import seaborn as sns

# =======================
# Image Processing
# =======================
from PIL import Image, ImageStat

# =======================
# Scikit-learn
# =======================
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error
)
from sklearn.utils import class_weight

# =======================
# Statistics
# =======================
from scipy.stats import pearsonr

# =======================
# TensorFlow / Keras
# =======================
import tensorflow as tf
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator,
    load_img,
    img_to_array
)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.utils import to_categorical

# =======================
# PyTorch
# =======================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# =======================
# Progress Bar
# =======================
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

### PreProcessing

#### FER-2013

In [ ]:
path_train = 'fer2013/train'
path_test = 'fer2013/test'

In [ ]:
class_names = sorted(os.listdir(path_train))
class_map = {class_name: i for i, class_name in enumerate(class_names)}
num_classes = len(class_names)
img_size = (224, 224, 3)

class_names

In [ ]:
def load_dataset(path):
  images_path = []
  labels = []

  for folder in os.listdir(path):
    label = folder
    for file in tqdm(os.listdir(os.path.join(path, folder)), desc = f"Loading {folder}"):
      img_path = os.path.join(path, folder, file)
      images_path.append(img_path)
      labels.append(label)

  return images_path, labels

In [ ]:
# convert dataset into data frame
images, labels = load_dataset(path_train)
train = pd.DataFrame({'images': list(images), 'labels': labels})

# shuffling the training dataset
train = train.sample(frac=1).reset_index(drop=True)

In [ ]:
test_images, test_labels = load_dataset(path_test)
test = pd.DataFrame({'images': list(test_images), 'labels': test_labels})

test = test.sample(frac=1).reset_index(drop=True)

In [ ]:
def show_raw_image(img_path):
    img = load_img(img_path)
    plt.imshow(img)
    plt.title("Raw Image")
    plt.axis("off")

show_raw_image(train['images'].iloc[5])

In [ ]:
# Feature Extraction
def extract_features(imgs):
  features = []
  for img in tqdm(imgs, desc = "Extracting features"):
    img = load_img(img, color_mode='rgb', target_size=(224, 224))
    img = img_to_array(img)

    features.append(img)
  return features

In [ ]:
train_features = extract_features(train['images'])
test_features = extract_features(test['images'])

In [ ]:
def show_resized_image(img_path):
    img = load_img(img_path, color_mode='rgb', target_size=(224, 224))
    img_array = img_to_array(img).astype("uint8")

    plt.imshow(img_array)
    plt.title("Resized Image (224x224)")
    plt.axis("off")

show_resized_image(train['images'].iloc[5])

In [ ]:
# Convert and Normalize
## Neural nets converge faster if pixel values are scaled to [0,1] (or standardized).
train_features = np.array(train_features) / 255.0
test_features = np.array(test_features) / 255.0

In [ ]:
def show_normalized_image(img_path):
    img = load_img(img_path, color_mode='rgb', target_size=(224, 224))
    img_array = img_to_array(img) / 255.0

    plt.imshow(img_array)
    plt.title("Normalized Image [0,1]")
    plt.axis("off")

    print("Before normalization:", img_to_array(load_img(train['images'].iloc[5])).min(),
      img_to_array(load_img(train['images'].iloc[5])).max())

    print("After normalization:", img_array.min(), img_array.max())

show_normalized_image(train['images'].iloc[5])

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(train_features)

In [ ]:
def show_augmentation(img_path, n=5):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, 0)

    aug_iter = datagen.flow(img_array, batch_size=1)

    plt.figure(figsize=(15,3))
    for i in range(n):
        augmented = next(aug_iter)[0].astype("uint8")
        plt.subplot(1, n, i+1)
        plt.imshow(augmented)
        plt.title(f"Augmented {i+1}")
        plt.axis("off")

show_augmentation(train['images'].iloc[5])

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(x=train['labels'])
plt.title("Class Distribution Before Balancing")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Balancing Dataset
# Convert labels to numerical format
y_train = train['labels'].map(class_map).values
y_test = test['labels'].map(class_map).values

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

In [ ]:
y_train = train['labels'].map(class_map).values
classes = np.unique(y_train)

inv_class_map = {v: k for k, v in class_map.items()}
class_names = [inv_class_map[c] for c in classes]

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

plt.figure(figsize=(10,6))
bars = plt.bar(class_names, class_weights)
plt.title("Class Weights Distribution")
plt.xlabel("Emotion Class")
plt.ylabel("Weight")
plt.xticks(rotation=45)
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width()/2,
        height,
        f"{height:.2f}",
        ha='center',
        va='bottom',
        fontsize=9
    )
plt.show()

In [ ]:
print("Train shape :", train_features.shape)
print("Test shape  :", test_features.shape)
print("Train label :", y_train.shape)
print("Test label  :", y_test.shape)

In [ ]:
raw_img = img_to_array(load_img(train['images'].iloc[0]))
norm_img = raw_img / 255.0

print("Mean pixel before normalization:", raw_img.mean())
print("Mean pixel after normalization :", norm_img.mean())

In [ ]:
# Save
np.savez_compressed(
    "fer2013_processed_final.npz",
    X_train=train_features,
    y_train=y_train,
    X_test=test_features,
    y_test=y_test
)

#### SCUT-FBP5500 V2

In [ ]:
scut_label_path= r'C:\Users\felic\OneDrive - Bina Nusantara\THESISSS\DATASET\SCUT-FBP5500\labels.txt'

In [ ]:
scut_df = pd.read_csv(scut_label_path, sep=' ', header=None, names=['filename', 'score'])

In [ ]:
scut_df['filename'] = scut_df['filename'].str.strip()

# Ambil daftar file yang benar-benar ada di folder
scut_image_folder = r'C:\Users\felic\OneDrive - Bina Nusantara\THESISSS\DATASET\SCUT-FBP5500\Images\Images'
available_images = set(os.listdir(scut_image_folder))

# Filter label hanya untuk file yang tersedia
scut_df_filtered = scut_df[scut_df['filename'].isin(available_images)]

print(f"Ada {len(scut_df_filtered)} data yang cocok antara label dan file gambar.")

In [ ]:
# Feature Extraction untuk SCUT-FBP5500 (tanpa brightness filter)
def extract_scut_features(df, image_folder):
    features = []
    labels = []
    failed = []
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting SCUT features"):
        img_path = os.path.join(image_folder, row['filename'].strip())
        try:
            img = Image.open(img_path).convert('RGB')

            # Resize ke 224x224
            img = img.resize((224, 224))
            img_array = np.array(img)
            features.append(img_array)
            labels.append(row['score'])
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            failed.append(row['filename'])
    return np.array(features), np.array(labels), failed

In [ ]:
scut_features, scut_labels, failed_files = extract_scut_features(scut_df_filtered, scut_image_folder)
print("File yang gagal dimuat:", failed_files)

In [ ]:
# Ekstraksi fitur
scut_features, scut_labels = extract_scut_features(scut_df_filtered, scut_image_folder)

In [ ]:
print(f"{len(scut_features)} gambar berhasil dimuat.")
print(f"Shape gambar: {scut_features.shape}")
print(f"Contoh label: {scut_labels[:5]}")

In [ ]:
# Convert dan Normalize
scut_features = scut_features / 255.0

In [ ]:
# Split 80/10/10 stratified
X_train, X_temp, y_train, y_temp = train_test_split(
    scut_features, scut_labels, test_size=0.2, stratify=np.round(scut_labels)
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=np.round(y_temp)
)

print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

In [ ]:
# Data Augmentation (ringan untuk SCUT)
scut_datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=10,          # rotasi kecil
    brightness_range=[0.8, 1.2] # penyesuaian brightness
)

scut_datagen.fit(X_train)

In [ ]:
# Save hasil preprocess ke file .npz
np.savez_compressed(
    "scutfbp5500_processed_final.npz",
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test
)
print("SCUT-FBP5500 dataset berhasil disimpan ke 'scutfbp5500_processed_final.npz'")

In [ ]:
images = []
labels = []

for idx, row in tqdm(df_filtered.iterrows(), total=len(df_filtered)):
    img_path = os.path.join(image_folder, row['filename'])
    try:
        img = Image.open(img_path).convert('RGB')
        img = img.resize((224, 224))  # Ukuran bisa disesuaikan dengan model
        img_array = np.array(img) / 255.0  # Normalisasi ke [0, 1]
        images.append(img_array)
        labels.append(row['score'])  # Sesuaikan dengan nama kolom label
    except Exception as e:
        print(f"Error loading {img_path}: {e}")

images = np.array(images)
labels = np.array(labels)

print(f"{len(images)} gambar berhasil dimuat.")
print(f"Shape gambar: {images.shape}")
print(f"Contoh label: {labels[:5]}")

### Modelling

#### Single-Task Beauty Score Regression

In [ ]:
class SCUTPreprocessedDataset(Dataset):
    """
    Dataset for pre-normalized SCUT-FBP5500 data
    Data is already normalized to [0, 1] and sized to 224x224
    """
    def __init__(self, npz_file, X_key, y_key, augment=False):
        print(f"Loading {X_key} and {y_key} from {npz_file}...")
        
        # Load data directly (it's small enough)
        data = np.load(npz_file)
        self.images = data[X_key]  # Shape: (N, 224, 224, 3), range [0, 1]
        self.scores = data[y_key].astype(np.float32)  # Shape: (N,)
        
        print(f"✓ Loaded {len(self.scores)} samples")
        print(f"✓ Image shape: {self.images.shape}")
        print(f"✓ Score range: {self.scores.min():.2f} to {self.scores.max():.2f}")
        
        self.augment = augment
        
        # Define augmentation transforms (if needed)
        if augment:
            self.aug_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                transforms.RandomRotation(degrees=5),
            ])
        
        # Always convert to tensor and apply ImageNet normalization
        self.to_tensor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                               std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.scores)
    
    def __getitem__(self, idx):
        # Get image (already normalized to [0,1])
        img_array = self.images[idx]  # Shape: (224, 224, 3), range [0, 1]
        score = float(self.scores[idx])
        
        # Convert to uint8 for PIL Image (denormalize to [0, 255])
        img_array = (img_array * 255).astype(np.uint8)
        
        # Convert to PIL Image
        img = Image.fromarray(img_array, mode='RGB')
        
        # Apply augmentation if training
        if self.augment:
            img = self.aug_transform(img)
        
        # Convert to tensor and normalize
        img = self.to_tensor(img)
        
        return img, score

In [ ]:
print("="*70)
print("LOADING SCUT-FBP5500 DATA")
print("="*70)

npz_file = 'scutfbp5500_processed_final.npz'

# Create datasets 
train_dataset = SCUTPreprocessedDataset(npz_file, 'X_train', 'y_train', augment=True)
val_dataset = SCUTPreprocessedDataset(npz_file, 'X_val', 'y_val', augment=False)
test_dataset = SCUTPreprocessedDataset(npz_file, 'X_test', 'y_test', augment=False)

print(f"\n✓ Train: {len(train_dataset)} samples")
print(f"✓ Val:   {len(val_dataset)} samples")
print(f"✓ Test:  {len(test_dataset)} samples")

batch_size = 32  # Adjust based on your GPU memory

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                         num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                       num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                        num_workers=0, pin_memory=True)

print(f"\n✓ DataLoaders created (batch_size={batch_size})")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n{'='*70}")
print(f"Using device: {device}")
if device.type == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"{'='*70}\n")

print("Loading EfficientNet-B0 for facial attractiveness regression...")

# Load pretrained EfficientNet-B0
# Gunakan weights parameter untuk PyTorch versi terbaru
try:
    # PyTorch >= 1.13 (gunakan weights)
    from torchvision.models import EfficientNet_B0_Weights
    model = models.efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)
except:
    # PyTorch < 1.13 (gunakan pretrained)
    model = models.efficientnet_b0(pretrained=True)

# Modify classifier for regression
# EfficientNet-B0 memiliki 1280 features di classifier
num_features = model.classifier[1].in_features

# Replace classifier dengan regression head
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(num_features, 512),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(512),
    nn.Dropout(p=0.3),
    nn.Linear(512, 128),
    nn.ReLU(inplace=True),
    nn.BatchNorm1d(128),
    nn.Dropout(p=0.2),
    nn.Linear(128, 1)  # Single output for regression
)

# Freeze early layers for transfer learning
# EfficientNet menggunakan 'features' untuk backbone
for name, param in model.named_parameters():
    # Hanya train classifier dan beberapa block terakhir
    if 'classifier' not in name and 'features.7' not in name and 'features.6' not in name:
        param.requires_grad = False

print("✓ EfficientNet-B0 modified for regression")
print(f"✓ Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"✓ Total parameters: {sum(p.numel() for p in model.parameters()):,}")

model = model.to(device)

In [ ]:
# Use MSE for regression
criterion = nn.MSELoss()

# Optimizer with different learning rates untuk berbagai layer
optimizer = optim.AdamW([
    {'params': model.classifier.parameters(), 'lr': 1e-3},
    {'params': model.features[7].parameters(), 'lr': 1e-4},
    {'params': model.features[6].parameters(), 'lr': 5e-5},
], weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5,
    min_lr=1e-7
)

# Mixed precision training
scaler = torch.cuda.amp.GradScaler() if device.type == "cuda" else None

# Training parameters
epochs = 100
patience = 15
min_delta = 0.001
best_val_mae = float('inf')
patience_counter = 0

history = {
    'train_loss': [], 'train_mae': [], 'train_rmse': [],
    'val_loss': [], 'val_mae': [], 'val_rmse': []
}

In [ ]:
print("\n" + "="*70)
print("SANITY CHECK")
print("="*70)

model.eval()
test_imgs, test_scores = next(iter(train_loader))
print(f"✓ Batch shape: {test_imgs.shape}")
print(f"✓ Batch min/max: {test_imgs.min():.4f}, {test_imgs.max():.4f}")
print(f"✓ Scores: {test_scores[:5].tolist()}")
print(f"✓ Scores range: {test_scores.min():.2f} to {test_scores.max():.2f}")

test_imgs = test_imgs.to(device)
with torch.no_grad():
    predictions = model(test_imgs).squeeze()

print(f"✓ Predictions shape: {predictions.shape}")
print(f"✓ Predictions sample: {predictions[:5].tolist()}")
print(f"✓ Predictions range: {predictions.min():.2f} to {predictions.max():.2f}")
print("\n✓ Model is working correctly!")
print("="*70 + "\n")

In [ ]:
print("="*70)
print("STARTING TRAINING WITH EFFICIENTNET-B0")
print("="*70 + "\n")

for epoch in range(epochs):
    epoch_start = time.time()
    
    # ============ TRAINING PHASE ============
    model.train()
    running_loss = 0.0
    train_preds, train_targets = [], []
    
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1:3d}/{epochs} [Train]", ncols=100)
    
    for imgs, scores in train_bar:
        imgs = imgs.to(device)
        scores = scores.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        
        if scaler:
            with torch.cuda.amp.autocast():
                outputs = model(imgs)
                loss = criterion(outputs, scores)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(imgs)
            loss = criterion(outputs, scores)
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item()
        train_preds.extend(outputs.detach().cpu().numpy())
        train_targets.extend(scores.cpu().numpy())
        
        train_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    train_preds = np.array(train_preds).flatten()
    train_targets = np.array(train_targets).flatten()
    train_mae = mean_absolute_error(train_targets, train_preds)
    train_rmse = np.sqrt(mean_squared_error(train_targets, train_preds))
    train_loss = running_loss / len(train_loader)
    
    # ============ VALIDATION PHASE ============
    model.eval()
    val_preds, val_targets = [], []
    val_loss = 0.0
    
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1:3d}/{epochs} [Val]  ", ncols=100)
        
        for imgs, scores in val_bar:
            imgs = imgs.to(device)
            scores = scores.to(device).float().unsqueeze(1)
            
            outputs = model(imgs)
            loss = criterion(outputs, scores)
            val_loss += loss.item()
            
            val_preds.extend(outputs.cpu().numpy())
            val_targets.extend(scores.cpu().numpy())
            
            val_bar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    val_preds = np.array(val_preds).flatten()
    val_targets = np.array(val_targets).flatten()
    val_mae = mean_absolute_error(val_targets, val_preds)
    val_rmse = np.sqrt(mean_squared_error(val_targets, val_preds))
    val_loss /= len(val_loader)
    
    # Pearson correlation
    val_corr = np.corrcoef(val_targets, val_preds)[0, 1]
    
    epoch_time = time.time() - epoch_start
    
    # Store history
    history['train_loss'].append(train_loss)
    history['train_mae'].append(train_mae)
    history['train_rmse'].append(train_rmse)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    history['val_rmse'].append(val_rmse)
    
    # Print summary
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1:3d}/{epochs} | Time: {epoch_time/60:.1f}min")
    print(f"{'-'*70}")
    print(f"Train | Loss: {train_loss:.4f} | MAE: {train_mae:.4f} | RMSE: {train_rmse:.4f}")
    print(f"Val   | Loss: {val_loss:.4f} | MAE: {val_mae:.4f} | RMSE: {val_rmse:.4f} | Corr: {val_corr:.4f}")
    print(f"{'='*70}")
    
    # Learning rate scheduling
    scheduler.step(val_mae)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Learning Rate: {current_lr:.2e}")
    
    # Early stopping
    if val_mae < best_val_mae - min_delta:
        best_val_mae = val_mae
        patience_counter = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_mae': val_mae,
            'val_rmse': val_rmse,
            'val_corr': val_corr,
            'history': history
        }, "best_efficientnet_scut_model.pth")
        print(f" Model saved! Best Val MAE: {val_mae:.4f}")
    else:
        patience_counter += 1
        print(f" No improvement ({patience_counter}/{patience})")
        
        if patience_counter >= patience:
            print(f"\n Early stopping at epoch {epoch+1}")
            print(f"Best Val MAE: {best_val_mae:.4f}")
            break
    print()

In [ ]:
print("\nPlotting training history...")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('Training & Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history['train_mae'], label='Train MAE', linewidth=2)
axes[1].plot(history['val_mae'], label='Val MAE', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Mean Absolute Error')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# RMSE
axes[2].plot(history['train_rmse'], label='Train RMSE', linewidth=2)
axes[2].plot(history['val_rmse'], label='Val RMSE', linewidth=2)
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('RMSE')
axes[2].set_title('Root Mean Squared Error')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('efficientnet_training_history.png', dpi=150, bbox_inches='tight')
print("✓ Training history saved as 'efficientnet_training_history.png'")

print("\n" + "="*70)
print("TRAINING COMPLETE!")
print("="*70)

In [ ]:
print("\n" + "="*70)
print("FINAL EVALUATION ON TEST SET")
print("="*70)

checkpoint = torch.load("best_efficientnet_scut_model.pth", weights_only=False)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"\n✓ Loaded best model from epoch {checkpoint['epoch']+1}")
print(f"✓ Best Val MAE: {checkpoint['val_mae']:.4f}")
print(f"✓ Best Val RMSE: {checkpoint['val_rmse']:.4f}")
print(f"✓ Best Val Correlation: {checkpoint['val_corr']:.4f}\n")

model.eval()

test_preds, test_targets = [], []
with torch.no_grad():
    for imgs, scores in tqdm(test_loader, desc="Testing"):
        imgs = imgs.to(device)
        outputs = model(imgs)
        test_preds.extend(outputs.cpu().numpy())
        test_targets.extend(scores.numpy())

test_preds = np.array(test_preds).flatten()
test_targets = np.array(test_targets).flatten()

# Calculate metrics
test_mae = mean_absolute_error(test_targets, test_preds)
test_rmse = np.sqrt(mean_squared_error(test_targets, test_preds))
test_mse = mean_squared_error(test_targets, test_preds)
test_corr = np.corrcoef(test_targets, test_preds)[0, 1]

print(f"{'='*70}")
print("TEST SET RESULTS:")
print(f"{'='*70}")
print(f"MAE:         {test_mae:.4f}")
print(f"RMSE:        {test_rmse:.4f}")
print(f"MSE:         {test_mse:.4f}")
print(f"Correlation: {test_corr:.4f}")
print(f"{'='*70}\n")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Scatter plot: Predictions vs Ground Truth
axes[0, 0].scatter(test_targets, test_preds, alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
axes[0, 0].plot([test_targets.min(), test_targets.max()], 
                [test_targets.min(), test_targets.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('True Attractiveness Score', fontsize=12)
axes[0, 0].set_ylabel('Predicted Score', fontsize=12)
axes[0, 0].set_title(f'Predictions vs Ground Truth\nMAE: {test_mae:.4f} | Corr: {test_corr:.4f}', 
                     fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Error distribution
errors = test_preds - test_targets
axes[0, 1].hist(errors, bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
axes[0, 1].set_xlabel('Prediction Error', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title(f'Error Distribution\nMean: {errors.mean():.4f} | Std: {errors.std():.4f}', 
                     fontsize=13, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Residual plot
axes[1, 0].scatter(test_targets, errors, alpha=0.6, s=30, edgecolors='black', linewidth=0.5)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('True Score', fontsize=12)
axes[1, 0].set_ylabel('Residual (Predicted - True)', fontsize=12)
axes[1, 0].set_title('Residual Plot', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Score distribution comparison
axes[1, 1].hist(test_targets, bins=20, alpha=0.5, label='True Scores', 
                edgecolor='black', color='green')
axes[1, 1].hist(test_preds, bins=20, alpha=0.5, label='Predicted Scores', 
                edgecolor='black', color='orange')
axes[1, 1].set_xlabel('Attractiveness Score', fontsize=12)
axes[1, 1].set_ylabel('Frequency', fontsize=12)
axes[1, 1].set_title('Score Distribution Comparison', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('efficientnet_scut_test_results.png', dpi=300, bbox_inches='tight')
print("✓ Test results saved as 'efficientnet_scut_test_results.png'")

#### Multi-Task Learning Beauty Score and Emotion Classification

In [ ]:
class EfficientNetMTLModel(nn.Module):
    """
    Multi-Task Learning with EfficientNet-B0 backbone
    Architecture matches thesis description exactly:
    - Feature Extractor: EfficientNet-B0 (pretrained)
    - Pooling Layer: Adaptive pooling for dimensionality reduction
    - Two branches: Regression (beauty) + Classification (emotion)
    """
    def __init__(self, num_emotions=7):
        super().__init__()
        
        # Feature Extractor: EfficientNet-B0 (pretrained on ImageNet)
        self.backbone = models.efficientnet_b0(pretrained=True)
        in_features = self.backbone.classifier[1].in_features  # 1280
        self.backbone.classifier = nn.Identity()  # Remove original classifier
        
        print(f"✓ Feature Extractor: EfficientNet-B0 (pretrained)")
        print(f"  Output features: {in_features}")
        
        # Shared neck for feature processing
        self.shared_neck = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # === REGRESSION BRANCH (Beauty Score Prediction) ===
        # Fully connected layers with dropout, ending in single neuron with linear activation
        self.regression_branch = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),  # Prevent overfitting
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)  # Single neuron, linear activation (implicit)
        )
        
        # === CLASSIFICATION BRANCH (Emotion Recognition) ===
        # Fully connected layers with dropout, ending in 7 neurons with softmax
        self.classification_branch = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),  # Prevent overfitting
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_emotions)  # 7 neurons for 7 emotion classes
        )
        
        print(f"✓ Regression Branch: Predicts beauty score [1.0 - 5.0]")
        print(f"✓ Classification Branch: Classifies 7 emotion categories")
        
    def forward(self, x):
        # EfficientNet sudah global avg pooling, output (B, 1280)
        features = self.backbone(x)          # (B, 1280)

        shared = self.shared_neck(features)  # (B, 512)

        beauty_score = self.regression_branch(shared).squeeze(1)  # (B,)
        emotion_logits = self.classification_branch(shared)       # (B, 7)

        return emotion_logits, beauty_score
    
    def predict(self, x):
        """
        Prediction with explicit softmax for inference
        (Thesis mentions softmax activation for probability distribution)
        """
        emotion_logits, beauty_score = self.forward(x)
        emotion_probs = torch.softmax(emotion_logits, dim=1)  # Explicit softmax
        return emotion_probs, beauty_score

In [ ]:
class FERDataset(Dataset):
    """FER-2013 Dataset"""
    def __init__(self, npz_path, split='train', transform=None):
        data = np.load(npz_path)
        
        if split == 'train':
            self.images = data['X_train']
            self.labels = data['y_train']
        else:
            self.images = data['X_test']
            self.labels = data['y_test']
        
        self.transform = transform
        print(f"✓ FER-2013 {split}: {len(self.labels)} samples")
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = int(self.labels[idx])
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        
        if self.transform:
            img = self.transform(img)
        
        return img, label

class SCUTDataset(Dataset):
    """SCUT-FBP5500 Dataset"""
    def __init__(self, npz_path, split='train', transform=None):
        data = np.load(npz_path)
        
        if split == 'train':
            self.images = data['X_train']
            self.scores = data['y_train']
        elif split == 'val':
            self.images = data['X_val']
            self.scores = data['y_val']
        else:
            self.images = data['X_test']
            self.scores = data['y_test']
        
        self.transform = transform
        self.score_min = 1.0
        self.score_max = 5.0
        
        print(f"✓ SCUT-FBP5500 {split}: {len(self.scores)} samples")
        
    def __len__(self):
        return len(self.scores)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        score = float(self.scores[idx])
        score_norm = (score - self.score_min) / (self.score_max - self.score_min)
        
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        
        if self.transform:
            img = self.transform(img)
        
        return img, score_norm

class CombinedMTLDataset(Dataset):
    """Combined dataset for MTL - FIXED VERSION"""
    def __init__(self, fer_dataset, scut_dataset, mode='alternate'):
        self.fer_dataset = fer_dataset
        self.scut_dataset = scut_dataset
        self.mode = mode
        
        self.fer_size = len(fer_dataset)
        self.scut_size = len(scut_dataset)
        
        if mode == 'alternate':
            # Create explicit index mapping to avoid duplicates
            self.indices = []
            fer_idx = 0
            scut_idx = 0
            
            # Alternate between FER and SCUT until both are exhausted
            while fer_idx < self.fer_size or scut_idx < self.scut_size:
                if fer_idx < self.fer_size:
                    self.indices.append(('fer', fer_idx))
                    fer_idx += 1
                
                if scut_idx < self.scut_size:
                    self.indices.append(('scut', scut_idx))
                    scut_idx += 1
            
            self.total_size = len(self.indices)
        else:
            # Simple concatenation
            self.total_size = self.fer_size + self.scut_size
        
        print(f"\n✓ Combined MTL Dataset ({mode} mode):")
        print(f"  FER: {self.fer_size} | SCUT: {self.scut_size} | Total: {self.total_size}")
        
    def __len__(self):
        return self.total_size
    
    def __getitem__(self, idx):
        if self.mode == 'alternate':
            # Use pre-computed index mapping
            source, source_idx = self.indices[idx]
            
            if source == 'fer':
                img, emotion_label = self.fer_dataset[source_idx]
                beauty_score = 0.0
                has_emotion = True
                has_beauty = False
            else:  # scut
                img, beauty_score = self.scut_dataset[source_idx]
                emotion_label = 0
                has_emotion = False
                has_beauty = True
        else:
            # Simple concatenation mode
            if idx < self.fer_size:
                img, emotion_label = self.fer_dataset[idx]
                beauty_score = 0.0
                has_emotion = True
                has_beauty = False
            else:
                img, beauty_score = self.scut_dataset[idx - self.fer_size]
                emotion_label = 0
                has_emotion = False
                has_beauty = True
        
        return img, emotion_label, beauty_score, has_emotion, has_beauty

In [ ]:
def train_mtl_epoch(model, train_loader, emotion_criterion, beauty_criterion,
                   optimizer, device, emotion_weight=1.0, beauty_weight=10.0):
    """Train one epoch"""
    model.train()
    
    total_loss = 0.0
    emotion_correct, emotion_total = 0, 0
    beauty_errors = []
    
    pbar = tqdm(train_loader, desc="Training")
    
    for images, emotion_labels, beauty_scores, has_emotion, has_beauty in pbar:
        images = images.to(device)
        emotion_labels = emotion_labels.to(device)
        beauty_scores = beauty_scores.to(device).float()
        has_emotion = has_emotion.to(device)
        has_beauty = has_beauty.to(device)
        
        optimizer.zero_grad()
        
        emotion_logits, beauty_pred = model(images)
        
        loss = 0.0
        
        if has_emotion.sum() > 0:
            emotion_loss = emotion_criterion(
                emotion_logits[has_emotion],
                emotion_labels[has_emotion]
            )
            loss += emotion_weight * emotion_loss
            
            _, preds = emotion_logits[has_emotion].max(1)
            emotion_correct += preds.eq(emotion_labels[has_emotion]).sum().item()
            emotion_total += has_emotion.sum().item()
        
        if has_beauty.sum() > 0:
            beauty_loss = beauty_criterion(
                beauty_pred[has_beauty],
                beauty_scores[has_beauty]
            )
            loss += beauty_weight * beauty_loss
            
            errors = torch.abs(beauty_pred[has_beauty] - beauty_scores[has_beauty])
            beauty_errors.extend(errors.detach().cpu().numpy())
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'emo': f'{100.*emotion_correct/max(emotion_total,1):.1f}%',
            'beauty': f'{np.mean(beauty_errors):.4f}' if beauty_errors else 'N/A'
        })
    
    avg_loss = total_loss / len(train_loader)
    train_emotion_acc = 100.0 * emotion_correct / max(emotion_total, 1)
    train_beauty_mae = np.mean(beauty_errors) if beauty_errors else 0.0
    
    return avg_loss, train_emotion_acc, train_beauty_mae

In [ ]:
def validate_mtl(model, fer_loader, scut_loader, device):
    """Validate on both tasks"""
    model.eval()
    
    # Emotion
    emotion_correct, emotion_total = 0, 0
    with torch.no_grad():
        for images, labels in fer_loader:
            images, labels = images.to(device), labels.to(device)
            emotion_logits, _ = model(images)
            _, preds = emotion_logits.max(1)
            emotion_correct += preds.eq(labels).sum().item()
            emotion_total += labels.size(0)
    
    emotion_acc = 100.0 * emotion_correct / emotion_total
    
    # Beauty
    beauty_preds, beauty_targets = [], []
    with torch.no_grad():
        for images, scores in scut_loader:
            images = images.to(device)
            _, beauty_pred = model(images)
            beauty_preds.extend(beauty_pred.cpu().numpy())
            beauty_targets.extend(scores.numpy())
    
    beauty_mae = mean_absolute_error(beauty_targets, beauty_preds)
    beauty_mae_original = beauty_mae * 4.0  # Denormalize
    
    return emotion_acc, beauty_mae, beauty_mae_original

In [ ]:
def train_efficientnet_mtl(fer_npz, scut_npz, device,
                          epochs=50, batch_size=64,
                          emotion_weight=1.0, beauty_weight=10.0):
    """
    Complete EfficientNet-B0 MTL training following thesis methodology:
    
    Transfer Learning Strategy (Thesis Section):
    1. Initial Training: Freeze all EfficientNet-B0 layers
       - Preserves low-level feature representations from pre-training
       - Only train classification and regression branches
    
    2. Fine-tuning: Gradual unfreezing (epoch 8)
       - Progressively unfreeze higher layers
       - Allows model to adapt to facial domain
    
    Optimization (Thesis Section):
    - Optimizer: AdamW (adaptive optimizer)
    - Learning rate: 2e-3 (optimal for stable convergence)
    - Batch size: 64 (balance between stability and memory efficiency)
    
    Regularization (Thesis Section):
    - Early stopping: Monitor validation loss
    - Dropout: Prevent overfitting (before output layers)
    
    Loss Function (Thesis Section):
    - Regression: MSE loss (measures difference between prediction and actual)
    - Classification: CrossEntropyLoss (measures probability distribution)
    - Total Loss: Weighted combination (emotion_weight + beauty_weight)
    """
    
    print("="*70)
    print("EFFICIENTNET-B0 MULTI-TASK LEARNING")
    print("="*70)
    
    # Load datasets
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
    ])
    
    fer_train = FERDataset(fer_npz, split='train', transform=train_transform)
    fer_test = FERDataset(fer_npz, split='test', transform=None)
    scut_train = SCUTDataset(scut_npz, split='train', transform=train_transform)
    scut_test = SCUTDataset(scut_npz, split='test', transform=None)
    
    combined_train = CombinedMTLDataset(fer_train, scut_train, mode='alternate')
    
    train_loader = DataLoader(combined_train, batch_size=batch_size,
                             shuffle=True, num_workers=0, pin_memory=True)
    fer_test_loader = DataLoader(fer_test, batch_size=batch_size,
                                 shuffle=False, num_workers=0)
    scut_test_loader = DataLoader(scut_test, batch_size=batch_size,
                                  shuffle=False, num_workers=0)
    
    # Model
    print("\n" + "="*70)
    print("CREATING EFFICIENTNET-B0 MODEL")
    print("="*70)
    
    model = EfficientNetMTLModel(num_emotions=7).to(device)
    
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"✓ Total parameters: {total_params:,}")
    print(f"✓ Trainable parameters: {trainable_params:,}")
    print(f"  (ResNet-50 has ~25.6M params)")
    print(f"  EfficientNet-B0 is {(25.6/total_params*1e6):.1f}x smaller! 🚀")
    
    # Loss & optimizer
    emotion_criterion = nn.CrossEntropyLoss()
    beauty_criterion = nn.MSELoss()
    
    # Freeze backbone initially
    for param in model.backbone.parameters():
        param.requires_grad = False
    print("\n✓ Backbone frozen for initial training")
    
    # Optimizer with higher learning rate (EfficientNet can handle it)
    optimizer = optim.AdamW([
        {'params': model.shared_neck.parameters(), 'lr': 2e-3},
        {'params': model.classification_branch.parameters(), 'lr': 2e-3},
        {'params': model.regression_branch.parameters(), 'lr': 2e-3},
    ], weight_decay=1e-4)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3
    )
    
    # Training loop
    best_emotion_acc = 0.0
    best_beauty_mae = float('inf')
    best_combined_score = -float('inf')
    patience_counter = 0
    patience = 10
    
    history = {
        'train_loss': [], 'train_emotion_acc': [], 'train_beauty_mae': [],
        'val_emotion_acc': [], 'val_beauty_mae': []
    }
    
    print("\n" + "="*70)
    print("STARTING TRAINING")
    print("="*70)
    print(f"Loss weights: emotion={emotion_weight}, beauty={beauty_weight}\n")
    
    for epoch in range(epochs):
        print(f"\n{'='*70}")
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"{'='*70}")
        
        # Train
        train_loss, train_emo_acc, train_beauty_mae = train_mtl_epoch(
            model, train_loader, emotion_criterion, beauty_criterion,
            optimizer, device, emotion_weight, beauty_weight
        )
        
        # Validate
        val_emo_acc, val_beauty_mae, val_beauty_mae_original = validate_mtl(
            model, fer_test_loader, scut_test_loader, device
        )
        
        # Store history
        history['train_loss'].append(train_loss)
        history['train_emotion_acc'].append(train_emo_acc)
        history['train_beauty_mae'].append(train_beauty_mae)
        history['val_emotion_acc'].append(val_emo_acc)
        history['val_beauty_mae'].append(val_beauty_mae_original)
        
        # Print
        print(f"\nResults:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Train Emotion: {train_emo_acc:.2f}%")
        print(f"  Train Beauty MAE: {train_beauty_mae:.4f}")
        print(f"  Val Emotion: {val_emo_acc:.2f}%")
        print(f"  Val Beauty MAE: {val_beauty_mae_original:.4f}")
        
        combined_score = (val_emo_acc / 100.0) - (val_beauty_mae_original / 5.0)
        
        scheduler.step(combined_score)
        
        # Unfreeze after epoch 8 (earlier than ResNet due to smaller model)
        if epoch == 8:
            print("\n Unfreezing backbone...")
            for param in model.backbone.parameters():
                param.requires_grad = True
            optimizer.add_param_group({'params': model.backbone.parameters(), 'lr': 5e-5})
        
        # Early stopping
        if combined_score > best_combined_score:
            best_combined_score = combined_score
            best_emotion_acc = val_emo_acc
            best_beauty_mae = val_beauty_mae_original
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'emotion_acc': val_emo_acc,
                'beauty_mae': val_beauty_mae_original,
                'history': history
            }, "mtl_efficientnet_best.pth")
            
            print(f"\n✅ New best! Emotion: {val_emo_acc:.2f}%, Beauty: {val_beauty_mae_original:.4f}")
        else:
            patience_counter += 1
            print(f"\n No improvement ({patience_counter}/{patience})")
            
            if patience_counter >= patience:
                print(f"\n Early stopping at epoch {epoch+1}")
                break
    
    print("\n" + "="*70)
    print("EFFICIENTNET-B0 TRAINING COMPLETE!")
    print("="*70)
    print(f"\nBest Results:")
    print(f"  Emotion Accuracy: {best_emotion_acc:.2f}%")
    print(f"  Beauty MAE: {best_beauty_mae:.4f}")
    print(f"\n Comparison to ResNet-50:")
    print(f"  ResNet-50:")
    print(f"    Emotion: 64.73% | Beauty MAE: 0.2187 | Params: 25.6M")
    print(f"  EfficientNet-B0:")
    print(f"    Emotion: {best_emotion_acc:.2f}% | Beauty MAE: {best_beauty_mae:.4f} | Params: ~5.3M")
    print(f"\n  Improvement:")
    print(f"    Emotion: {best_emotion_acc-64.73:+.2f}%")
    print(f"    Beauty MAE: {best_beauty_mae-0.2187:+.4f}")
    print(f"    Parameters: -79% smaller! ")
    print("="*70)
    
    return history, model

In [ ]:
if __name__ == "__main__":
    # Paths
    FER_data = "fer2013_processed_final.npz"
    SCUT_data = "scutfbp5500_processed_final.npz"
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")
    
    # Train
    history, model = train_efficientnet_mtl(
        fer_npz=FER_data,
        scut_npz=SCUT_data,
        device=device,
        epochs=50,
        batch_size=64,
        emotion_weight=1.0,
        beauty_weight=10.0
    )
    
    print("\n✓ Model saved as: mtl_efficientnet_best.pth")
    print("✓ Ready for evaluation!")

In [ ]:
class EfficientNetMTLModel(nn.Module):
    """Same architecture as training"""
    def __init__(self, num_emotions=7):
        super().__init__()
        
        self.backbone = models.efficientnet_b0(pretrained=True)
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Identity()
    
        self.shared_neck = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        self.regression_branch = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )
        
        self.classification_branch = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_emotions)
        )
        
    def forward(self, x):
        features = self.backbone(x)
        shared = self.shared_neck(features)
        
        beauty_score = self.regression_branch(shared).squeeze(1)
        emotion_logits = self.classification_branch(shared)
        
        return emotion_logits, beauty_score
    
    def predict(self, x):
        """Prediction with explicit softmax"""
        emotion_logits, beauty_score = self.forward(x)
        emotion_probs = torch.softmax(emotion_logits, dim=1)
        return emotion_probs, beauty_score

def load_model(checkpoint_path, device):
    """Load trained EfficientNet MTL model"""
    model = EfficientNetMTLModel(num_emotions=7).to(device)
    
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"✓ Loaded EfficientNet-B0 MTL model from epoch {checkpoint['epoch']+1}")
    print(f"✓ Best Emotion Acc: {checkpoint['emotion_acc']:.2f}%")
    print(f"✓ Best Beauty MAE: {checkpoint['beauty_mae']:.4f}")
    
    return model, checkpoint


class FERDataset:
    """FER-2013 Dataset"""
    def __init__(self, npz_path, split='test'):
        data = np.load(npz_path)
        self.images = data['X_test']
        self.labels = data['y_test']
        print(f"✓ FER-2013 {split}: {len(self.labels)} samples")
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        label = int(self.labels[idx])
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        return img, label

class SCUTDataset:
    """SCUT-FBP5500 Dataset"""
    def __init__(self, npz_path, split='test'):
        data = np.load(npz_path)
        self.images = data['X_test']
        self.scores = data['y_test']
        print(f"✓ SCUT-FBP5500 {split}: {len(self.scores)} samples")
        
    def __len__(self):
        return len(self.scores)
    
    def __getitem__(self, idx):
        img = self.images[idx]
        score = float(self.scores[idx])
        score_norm = (score - 1.0) / 4.0
        img = torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)
        return img, score_norm


def evaluate_emotion(model, test_loader, device):
    """Evaluate emotion classification"""
    model.eval()
    
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            emotion_logits, _ = model(images)
            
            probs = torch.softmax(emotion_logits, dim=1)
            _, preds = emotion_logits.max(1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    accuracy = accuracy_score(all_labels, all_preds)
    
    return all_preds, all_labels, all_probs, accuracy

def evaluate_beauty(model, test_loader, device):
    """Evaluate beauty regression"""
    model.eval()
    
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for images, scores in test_loader:
            images = images.to(device)
            _, beauty_pred = model(images)
            
            # Denormalize to [1, 5]
            beauty_pred_original = beauty_pred.cpu().numpy() * 4.0 + 1.0
            scores_original = scores.numpy() * 4.0 + 1.0
            
            all_preds.extend(beauty_pred_original)
            all_targets.extend(scores_original)
    
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    
    mae = mean_absolute_error(all_targets, all_preds)
    rmse = np.sqrt(np.mean((all_targets - all_preds) ** 2))
    correlation, _ = pearsonr(all_targets, all_preds)
    
    return all_preds, all_targets, mae, rmse, correlation


def plot_emotion_confusion_matrix(labels, preds, save_path='efficientnet_emotion_cm.png'):
    """Plot confusion matrix for emotion"""
    emotion_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    cm = confusion_matrix(labels, preds)
    accuracy = accuracy_score(labels, preds)
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Counts
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_names, yticklabels=emotion_names,
                cbar_kws={'label': 'Count'}, ax=axes[0])
    axes[0].set_xlabel('Predicted Emotion', fontsize=12)
    axes[0].set_ylabel('True Emotion', fontsize=12)
    axes[0].set_title(f'EfficientNet-B0 Confusion Matrix (Counts)\nAccuracy: {accuracy:.2%}', 
                     fontsize=14, fontweight='bold')
    
    # Percentages
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=emotion_names, yticklabels=emotion_names,
                cbar_kws={'label': 'Percentage'}, ax=axes[1])
    axes[1].set_xlabel('Predicted Emotion', fontsize=12)
    axes[1].set_ylabel('True Emotion', fontsize=12)
    axes[1].set_title('Normalized Confusion Matrix', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()

def plot_beauty_results(targets, preds, mae, rmse, corr, save_path='efficientnet_beauty_results.png'):
    """Plot beauty regression results"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Scatter plot
    axes[0].scatter(targets, preds, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
    axes[0].plot([1, 5], [1, 5], 'r--', linewidth=2, label='Perfect Prediction')
    axes[0].set_xlabel('True Beauty Score', fontsize=12)
    axes[0].set_ylabel('Predicted Score', fontsize=12)
    axes[0].set_title(f'EfficientNet-B0 Beauty Predictions\nMAE: {mae:.4f} | Corr: {corr:.4f}',
                     fontsize=13, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim(0.5, 5.5)
    axes[0].set_ylim(0.5, 5.5)
    
    # Error distribution
    errors = preds - targets
    axes[1].hist(errors, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Zero Error')
    axes[1].set_xlabel('Prediction Error', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title(f'Error Distribution\nMean: {errors.mean():.4f} | Std: {errors.std():.4f}',
                     fontsize=13, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    
    # Residual plot
    axes[2].scatter(targets, errors, alpha=0.5, s=30, edgecolors='black', linewidth=0.5)
    axes[2].axhline(0, color='red', linestyle='--', linewidth=2)
    axes[2].set_xlabel('True Score', fontsize=12)
    axes[2].set_ylabel('Residual', fontsize=12)
    axes[2].set_title('Residual Plot', fontsize=13, fontweight='bold')
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()

def plot_per_class_accuracy(labels, preds, save_path='efficientnet_per_class_acc.png'):
    """Plot per-class emotion accuracy"""
    emotion_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    class_accs = []
    class_counts = []
    
    for i in range(7):
        mask = labels == i
        if mask.sum() > 0:
            acc = accuracy_score(labels[mask], preds[mask])
            class_accs.append(acc * 100)
            class_counts.append(mask.sum())
        else:
            class_accs.append(0)
            class_counts.append(0)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(emotion_names))
    bars = ax.bar(x, class_accs, color='steelblue', edgecolor='black', alpha=0.7)
    
    # Add labels
    for i, (bar, count) in enumerate(zip(bars, class_counts)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
                f'{height:.1f}%\n(n={count})',
                ha='center', va='bottom', fontsize=9)
    
    ax.set_xlabel('Emotion Class', fontsize=12)
    ax.set_ylabel('Accuracy (%)', fontsize=12)
    ax.set_title('EfficientNet-B0 Per-Class Emotion Accuracy', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(emotion_names, rotation=45, ha='right')
    ax.set_ylim(0, 100)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()

def visualize_sample_predictions(model, fer_dataset, scut_dataset, device,
                                n_samples=8, save_path='efficientnet_samples.png'):
    """Visualize sample predictions"""
    model.eval()
    emotion_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    
    fig, axes = plt.subplots(2, n_samples, figsize=(3*n_samples, 7))
    
    # FER samples
    fer_indices = random.sample(range(len(fer_dataset)), n_samples)
    for i, idx in enumerate(fer_indices):
        img_tensor, true_label = fer_dataset[idx]
        
        with torch.no_grad():
            img_batch = img_tensor.unsqueeze(0).to(device)
            emotion_logits, beauty_pred = model(img_batch)
            probs = torch.softmax(emotion_logits, dim=1)
            pred_label = emotion_logits.argmax(dim=1).item()
            confidence = probs[0, pred_label].item()
            beauty_score = beauty_pred.item() * 4.0 + 1.0
        
        img_display = img_tensor.permute(1, 2, 0).cpu().numpy()
        img_display = np.clip(img_display, 0, 1)
        
        axes[0, i].imshow(img_display)
        axes[0, i].axis('off')
        
        color = 'green' if pred_label == true_label else 'red'
        axes[0, i].set_title(
            f"True: {emotion_names[true_label]}\n"
            f"Pred: {emotion_names[pred_label]}\n"
            f"Conf: {confidence:.2%}\n"
            f"Beauty: {beauty_score:.2f}",
            fontsize=9, color=color, fontweight='bold'
        )
    
    # SCUT samples
    scut_indices = random.sample(range(len(scut_dataset)), n_samples)
    for i, idx in enumerate(scut_indices):
        img_tensor, true_score_norm = scut_dataset[idx]
        true_score = true_score_norm * 4.0 + 1.0
        
        with torch.no_grad():
            img_batch = img_tensor.unsqueeze(0).to(device)
            emotion_logits, beauty_pred = model(img_batch)
            probs = torch.softmax(emotion_logits, dim=1)
            pred_emotion = emotion_logits.argmax(dim=1).item()
            emotion_conf = probs[0, pred_emotion].item()
            pred_score = beauty_pred.item() * 4.0 + 1.0
        
        img_display = img_tensor.permute(1, 2, 0).cpu().numpy()
        img_display = np.clip(img_display, 0, 1)
        
        axes[1, i].imshow(img_display)
        axes[1, i].axis('off')
        
        error = abs(pred_score - true_score)
        color = 'green' if error < 0.3 else 'orange' if error < 0.6 else 'red'
        axes[1, i].set_title(
            f"True: {true_score:.2f}\n"
            f"Pred: {pred_score:.2f}\n"
            f"Error: {error:.2f}\n"
            f"Emotion: {emotion_names[pred_emotion]}",
            fontsize=9, color=color, fontweight='bold'
        )
    
    fig.text(0.01, 0.75, 'FER-2013\n(Emotion)', 
             fontsize=14, fontweight='bold', rotation=90, va='center')
    fig.text(0.01, 0.25, 'SCUT-FBP5500\n(Beauty)', 
             fontsize=14, fontweight='bold', rotation=90, va='center')
    
    plt.suptitle('EfficientNet-B0 Sample Predictions', fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0.03, 0, 1, 0.97])
    plt.savefig(save_path, dpi=200, bbox_inches='tight')
    print(f"✓ Saved: {save_path}")
    plt.close()


def evaluate_efficientnet_mtl(checkpoint_path, fer_npz, scut_npz, device):
    """Complete evaluation pipeline"""
    
    print("="*70)
    print("EFFICIENTNET-B0 MTL MODEL EVALUATION")
    print("="*70 + "\n")
    
    # Load model
    model, checkpoint = load_model(checkpoint_path, device)
    
    # Load datasets
    print("\n📂 Loading test datasets...")
    fer_test = FERDataset(fer_npz, split='test')
    scut_test = SCUTDataset(scut_npz, split='test')
    
    fer_test_loader = DataLoader(fer_test, batch_size=64, shuffle=False, num_workers=0)
    scut_test_loader = DataLoader(scut_test, batch_size=64, shuffle=False, num_workers=0)
    
    # Evaluate emotion
    print("\n" + "="*70)
    print("EVALUATING EMOTION CLASSIFICATION")
    print("="*70 + "\n")
    
    emotion_preds, emotion_labels, emotion_probs, emotion_acc = evaluate_emotion(
        model, fer_test_loader, device
    )
    
    print(f"Emotion Accuracy: {emotion_acc:.2%} ({emotion_acc*100:.2f}%)")
    
    emotion_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    print("\nClassification Report:")
    print(classification_report(emotion_labels, emotion_preds, target_names=emotion_names))
    
    print("\nPer-Class Accuracy:")
    for i, name in enumerate(emotion_names):
        mask = emotion_labels == i
        if mask.sum() > 0:
            acc = accuracy_score(emotion_labels[mask], emotion_preds[mask])
            print(f"  {name:12s}: {acc:.2%} ({mask.sum()} samples)")
    
    # Evaluate beauty
    print("\n" + "="*70)
    print("EVALUATING BEAUTY REGRESSION")
    print("="*70 + "\n")
    
    beauty_preds, beauty_targets, beauty_mae, beauty_rmse, beauty_corr = evaluate_beauty(
        model, scut_test_loader, device
    )
    
    print(f"Beauty MAE:  {beauty_mae:.4f}")
    print(f"Beauty RMSE: {beauty_rmse:.4f}")
    print(f"Pearson Correlation: {beauty_corr:.4f}")
    
    # Generate visualizations
    print("\n" + "="*70)
    print("GENERATING VISUALIZATIONS")
    print("="*70 + "\n")
    
    plot_emotion_confusion_matrix(emotion_labels, emotion_preds)
    plot_per_class_accuracy(emotion_labels, emotion_preds)
    plot_beauty_results(beauty_targets, beauty_preds, beauty_mae, beauty_rmse, beauty_corr)
    visualize_sample_predictions(model, fer_test, scut_test, device)
    
    # Final summary
    print("\n" + "="*70)
    print("FINAL RESULTS SUMMARY")
    print("="*70)
    print(f"\n Emotion Classification:")
    print(f"   Test Accuracy: {emotion_acc:.2%}")
    print(f"\n Beauty Regression:")
    print(f"   Test MAE: {beauty_mae:.4f}")
    print(f"   Test RMSE: {beauty_rmse:.4f}")
    print(f"   Correlation: {beauty_corr:.4f}")
    print(f"\n Generated Files:")
    print(f"   - efficientnet_emotion_cm.png")
    print(f"   - efficientnet_per_class_acc.png")
    print(f"   - efficientnet_beauty_results.png")
    print(f"   - efficientnet_samples.png")
    print("="*70)
    
    return {
        'emotion_acc': emotion_acc,
        'beauty_mae': beauty_mae,
        'beauty_rmse': beauty_rmse,
        'beauty_corr': beauty_corr
    }


if __name__ == "__main__":
    # Paths
    checkpoint_path = "mtl_efficientnet_best.pth"
    fer_npz = "fer2013_processed_final.npz"
    scut_npz = "scutfbp5500_processed_final.npz"
    
    # Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")
    
    # Run evaluation
    results = evaluate_efficientnet_mtl(checkpoint_path, fer_npz, scut_npz, device)
    
    print("\n✓ Evaluation complete!")
    print(f"\nQuick Summary:")
    print(f"  Emotion: {results['emotion_acc']*100:.2f}%")
    print(f"  Beauty MAE: {results['beauty_mae']:.4f}")

In [6]:
import streamlit as st

In [7]:
@st.cache_resource
def load_models():
    """Load both models"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    models_dict = {
        'beauty': None,
        'mtl': None
    }
    
    # Load Beauty Regression Model
    try:
        beauty_model = BeautyRegressionModel()
        checkpoint = torch.load('best_efficientnet_scut_model.pth', 
                          map_location=device, weights_only=False)  # Changed to False
    
        # Load the model state dict properly
        if 'model_state_dict' in checkpoint:
            beauty_model.load_state_dict(checkpoint['model_state_dict'], strict=True)
        else:
            beauty_model.load_state_dict(checkpoint, strict=True)
    
        beauty_model.to(device)
        beauty_model.eval()
        models_dict['beauty'] = beauty_model
        st.success("✅ Beauty regression model loaded successfully")
    except FileNotFoundError:
        st.warning("⚠️ Beauty regression model file not found: 'best_efficientnet_scut_model.pth'")
    except Exception as e:
        st.error(f"❌ Error loading beauty model: {str(e)[:200]}")
    
    # Load Multi-Task Learning Model
    try:
        mtl_model = EfficientNetMTLModel(num_emotions=7)
        checkpoint = torch.load('mtl_efficientnet_best.pth', 
                              map_location=device, weights_only=False)  # Changed to False
        mtl_model.load_state_dict(checkpoint['model_state_dict'])
        mtl_model.to(device)
        mtl_model.eval()
        models_dict['mtl'] = mtl_model
        st.success("✅ Multi-task model loaded successfully")
    except FileNotFoundError:
        st.warning("⚠️ Multi-task model file not found: 'mtl_efficientnet_best.pth'")
    except Exception as e:
        st.error(f"❌ Error loading MTL model: {str(e)[:200]}")
    
    return models_dict, device

In [8]:
checkpoint = torch.load('best_efficientnet_scut_model.pth', 
                       map_location='cpu', weights_only=False)

print("Keys in model_state_dict:")
for key in list(checkpoint['model_state_dict'].keys())[:10]:
    print(f"  {key}")

Keys in model_state_dict:
  features.0.0.weight
  features.0.1.weight
  features.0.1.bias
  features.0.1.running_mean
  features.0.1.running_var
  features.0.1.num_batches_tracked
  features.1.0.block.0.0.weight
  features.1.0.block.0.1.weight
  features.1.0.block.0.1.bias
  features.1.0.block.0.1.running_mean
